<a href="https://colab.research.google.com/github/Farouk96/Predict_tags_question/blob/main/Questions__tags_mod%C3%A9lisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
from matplotlib import pyplot
import pickle

import time

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

#One Vs rest
from sklearn.multiclass import OneVsRestClassifier

from sklearn.linear_model import Perceptron,LogisticRegression,LogisticRegressionCV,SGDClassifier,PassiveAggressiveClassifier

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn.svm import LinearSVC 


#Multioutput
from sklearn.multioutput import MultiOutputClassifier

from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier

from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier



#Score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import f1_score, make_scorer

from sklearn.model_selection import GridSearchCV

from scipy.sparse import hstack

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Questions_Tags_Exploration.csv', sep=',')

# Data preparation


*   Binariser les tags
*   Utiliser un TFIDF pour body et Title Les paramètres du TFIDF sont très importants pour la performance de nos tags puisque nous ne voulons pas qu'il supprime des mots comme c# ou .net. Pour cela, nous devons utiliser le motif suivant : token_pattern=r"(?u)\S\S+"


In [ ]:
X1 = data['Body']
X2 = data['Title']
y = data['Tags']

In [ ]:
multilabel_binarizer = MultiLabelBinarizer()
y_bin = multilabel_binarizer.fit_transform(y)

In [ ]:
vectorizer_X1 = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       token_pattern=r"(?u)\S\S+",
                                       max_features=5000)

vectorizer_X2 = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       token_pattern=r"(?u)\S\S+",
                                       max_features=5000)

In [ ]:
X1_tfidf = vectorizer_X1.fit_transform(X1)
X2_tfidf = vectorizer_X2.fit_transform(X2)

In [ ]:
X_tfidf = hstack([X1_tfidf,X2_tfidf])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_bin, test_size = 0.2, random_state = 0) # Do 80/20 split
y_train.shape

(9784, 39)

#Modélisation supérvisé

In [ ]:
def avg_jacard(y_true,y_pred):
    '''
    see https://en.wikipedia.org/wiki/Multi-label_classification#Statistics_and_evaluation_metrics
    '''
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    
    return jacard.mean()*100
def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
    print("F1-score: {}".format(f1_score(y_test, y_pred,average='micro')))
    print("---")  

##One vs Rest
Pour évaluer nos modèles, j'utiliserai **le score de Jacard** car il est le mieux adapté à la classification multi-étiquettes.

In [ ]:
perceptron=Perceptron()
lr=LogisticRegression()
lr_cv=LogisticRegressionCV()
sgd=SGDClassifier()
pass_agr=PassiveAggressiveClassifier()

#boost=GradientBoostingClassifier()

gauss= GaussianProcessClassifier()

svc=LinearSVC() 


for classifier in [pass_agr, sgd, lr, svc, perceptron]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score(y_pred, classifier)

/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 2 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 10 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 11 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 12 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])


Clf:  PassiveAggressiveClassifier
Jacard score: 66.06878829212415
F1-score: 0.7881281955356029
---


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 2 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 10 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 11 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 12 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])


Clf:  SGDClassifier
Jacard score: 71.07480925822716
F1-score: 0.8214647968958799
---


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 2 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 10 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 11 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 12 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])


Clf:  LogisticRegression
Jacard score: 70.16204722883528
F1-score: 0.8148087125580782
---


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 2 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 10 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 11 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 12 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])


Clf:  LinearSVC
Jacard score: 69.67745115726837
F1-score: 0.8122020060276506
---


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 2 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 10 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 11 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 12 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])


Clf:  Perceptron
Jacard score: 65.82849346931233
F1-score: 0.7870110105135651
---


##Multi output

In [ ]:
DT=DecisionTreeClassifier()
ETC=ExtraTreeClassifier()

KNC=KNeighborsClassifier()
RNC=RadiusNeighborsClassifier()

RF=RandomForestClassifier()

In [ ]:
for classifier in [DT,ETC,KNC,RF]:
    clf = MultiOutputClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score(y_pred, classifier)

Clf:  DecisionTreeClassifier
Jacard score: 63.6376505714749
F1-score: 0.7760038403766047
---
Clf:  ExtraTreeClassifier
Jacard score: 58.61294361425414
F1-score: 0.7389861147906311
---
Clf:  KNeighborsClassifier
Jacard score: 54.245417173556085
F1-score: 0.6876184920519177
---
Clf:  RandomForestClassifier
Jacard score: 71.54269696236457
F1-score: 0.8265854532425654
---


##GridSearch CV on the best classifier

In [ ]:
param_grid = {'estimator__penalty':["l2", "l1", "elasticnet"]
              }     

In [ ]:
sgd = OneVsRestClassifier(SGDClassifier())
CV_sgd = GridSearchCV(estimator=sgd, param_grid=param_grid, cv= 5,verbose=10, scoring=make_scorer(avg_jacard,greater_is_better=True))
CV_sgd.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5; 1/3] START estimator__penalty=l2.......................................


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 2 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 10 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 11 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 12 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])


[CV 1/5; 1/3] END .......estimator__penalty=l2;, score=70.159 total time=   1.5s
[CV 2/5; 1/3] START estimator__penalty=l2.......................................


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 2 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 10 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 11 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 12 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])


[CV 2/5; 1/3] END .......estimator__penalty=l2;, score=70.305 total time=   1.6s
[CV 3/5; 1/3] START estimator__penalty=l2.......................................


...

[CV 4/5; 3/3] END estimator__penalty=elasticnet;, score=71.524 total time=   5.8s
[CV 5/5; 3/3] START estimator__penalty=elasticnet...............................


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 2 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 10 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 11 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 12 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])


[CV 5/5; 3/3] END estimator__penalty=elasticnet;, score=70.672 total time=   5.7s


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 2 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 10 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 11 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label 12 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])


GridSearchCV(cv=5, estimator=OneVsRestClassifier(estimator=SGDClassifier()),
             param_grid={'estimator__penalty': ['l2', 'l1', 'elasticnet']},
             scoring=make_scorer(avg_jacard), verbose=10)

In [ ]:
CV_sgd.best_params_

{'estimator__penalty': 'l1'}

In [ ]:
CV_sgd.best_estimator_

OneVsRestClassifier(estimator=SGDClassifier(penalty='l1'))

In [ ]:
best_model = CV_sgd.best_estimator_
y_pred = best_model.predict(X_test)

print_score(y_pred, best_model)

Clf:  OneVsRestClassifier
Jacard score: 71.45555730483204
F1-score: 0.8238150706281889
---


In [ ]:
with open('Best tager questions','wb') as f:
  pickle.dump(best_model,f)

##Confusion matrix

In [ ]:
y_train.shape[1]

39

In [ ]:
for i in range(y_train.shape[1]):
    print(multilabel_binarizer.classes_[i])
    print(confusion_matrix(y_test[:,i], y_pred[:,i]))
    print("")

NameError: ignored

##Exctracting feature importance

In [ ]:
def print_top10(feature_names, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.coef_[i])[-10:]
        print("--------------------------------------------")
        print("%s: %s" % (class_label,
              " ".join(feature_names[j] for j in top10)))
        print("--------------------------------------------")

In [ ]:
feature_names = vectorizer_X1.get_feature_names() + vectorizer_X2.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print_top10(feature_names, best_model, multilabel_binarizer.classes_)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26). If you observe this warning while using RFE or SelectFromModel, use the importance_getter parameter instead.
  warnings.warn(msg, category=FutureWarning)


AttributeError: ignored

In [ ]:
best_model.coef_

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26). If you observe this warning while using RFE or SelectFromModel, use the importance_getter parameter instead.
  warnings.warn(msg, category=FutureWarning)


AttributeError: ignored